In [15]:
import numpy as np
import paint_auth as pa
import pandas as pd
import tensorflow as tf
import resnet
import glob
from lxml import etree as ET
from collections import Counter
from sklearn.model_selection import train_test_split

In [16]:
#import all xml and convert to pd dataframe
#only extract painters
path = '/Users/Mason/Desktop/adv_big_data/data/xml2'
file_names = glob.glob(path+'/**')

#initialize dict and parser
info = {}
parser = ET.XMLParser(recover=True)
for i in range(len(file_names)):
    im_name = file_names[i].split('/')[-1]
    tree = ET.parse(file_names[i], parser)
    title, artist, medium, period = pa.extract_info(tree.getroot())
    if medium == 'schilderij' or medium == 'tekening': #painting or drawing in Dutch...
        if artist is not None:
            artist = artist.split(': ')[-1]
        im_name = im_name.split('.xml')[0]
        info[i] = (im_name, title, artist, medium, period)

info = pd.DataFrame.from_dict(info, 'index').rename(index=str, columns={0:'file_name', 1:'title', 2:'artist', 3:'medium', 4:'period'})

In [17]:
#counting and sorting artists' number of paintings
counts = Counter(info['artist'].tolist())
counts = pd.DataFrame.from_dict(counts, orient='index').sort_values(by=0,ascending=False)
#getting subset of artists with decent number of paintings
my_artists = counts.index[1:6].tolist()
my_artists = info[info['artist'].isin(my_artists)]

In [18]:
image_path = '/Users/Mason/Desktop/adv_big_data/data/jpg2'
paintings, mult, one_hot = pa.import_dataset(image_path, my_artists)

imported images: 100
imported images: 200
imported images: 300
imported images: 400
imported images: 500
imported images: 600
imported images: 700
imported images: 800
imported images: 900
imported images: 1000
imported images: 1100
imported images: 1200
imported images: 1300
imported images: 1400


In [19]:
#train/test split
X_train, X_test, y_train, y_test = train_test_split(paintings, one_hot, test_size=0.2, stratify=one_hot)

In [20]:
#normalize training
import cv2
t = cv2.normalize(X_train[0].astype('float32'), None, 0.0, 1.0, cv2.NORM_MINMAX)
X_train = np.array([cv2.normalize(image.astype('float32'), None, 0.0, 1.0, cv2.NORM_MINMAX) for image in X_train])

In [22]:
X_train.shape

(2088, 256, 256, 3)

In [23]:
#initializing Resnet
#hyperparams
alpha = 1e-2
BATCH_SIZE = 100
n_classes = len(y_train[0])
shape = (256,256,3)
optimizer = tf.train.AdamOptimizer(alpha)
sess = tf.Session()
model = resnet.Resnet(shape, n_classes, optimizer, sess)
model.build_model()
model.build_loss()

(?, 5)


In [24]:
#test run
sess.run(tf.global_variables_initializer())
model.predict([X_train[0]])

array([[0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
train_acc = []
BATCH_SIZE = 100
#training
count = 0
for offset in range(0, len(y_train), BATCH_SIZE):
    print 'beginning training for iteration: %d' % count
    batch_X = X_train[offset: offset + BATCH_SIZE]
    batch_y = y_train[offset: offset + BATCH_SIZE]
    
    model.train(batch_X, batch_y)
    y_hat = model.predict(batch_X)
    #retrieving labels
    y_hat_labels = np.argmax(y_hat)
    y_labels = np.argmax(batch_y, 1)
    
    acc = np.mean((y_labels-y_hat_labels)==0)
    print 'train acc.: %f' % acc
    train_acc.append(acc)
    count += 1

beginning training for iteration: 0
train acc.: 0.150000
beginning training for iteration: 1
train acc.: 0.170000
beginning training for iteration: 2
train acc.: 0.100000
beginning training for iteration: 3
train acc.: 0.220000
beginning training for iteration: 4
train acc.: 0.240000
beginning training for iteration: 5
train acc.: 0.340000
beginning training for iteration: 6


In [203]:
X_train[0]

<tf.Tensor 'div_3845:0' shape=(256, 256, 3) dtype=float32>